In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

크로스 밸리데이션  
앞선 지도 학습 알고리즘에서는 전체 데이터를 트레이닝 데이터와 테스트 데이터로 나누어 모형을 학습시켰다. 이번에는 하이퍼파라미터 튜닝을 위해 크로스 밸리데이션 기법을 활용해본다.

<table align="left" width="700">
    <tr>
        <td colspan="6" style="border: 1px solid;">
            <div style="text-align: center">Total Data</div>
        </td>
    </tr>
    <tr>
        <td colspan="5" style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td rowspan="5" style="border: 1px solid;">
            <div style="text-align: center">
                파라미터,<br>
                하이퍼파라미터<br>
                설정
            </div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td colspan="5">
            <div>모형평가</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
</table>

In [2]:
# 데이터 불러오기
from sklearn import datasets # 와인 데이터를 사용하기 위해 import 한다.
raw_wine = datasets.load_wine() # 와인 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_wine.data # 와인 피쳐 데이터를 저장한다.
y = raw_wine.target # 와인 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝/테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0) # 트레이닝 데이터와 테스트 데이터로 분할한다.
print(X_tn.shape, X_te.shape)

(133, 13) (45, 13)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn) # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
from sklearn import svm # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
# stratified K-Fold cross validation은 일반적인 K-Fold cross validation과는 달리 라벨링 비율을 유지하면서 데이터를
# 추출한다.
# stratified K-Fold cross validation 알고리즘을 사용하기 위해 import 한다.
from sklearn.model_selection import StratifiedKFold
clf_svm = svm.SVC(random_state=0) # 서포트 벡터 머신 모델 객체를 만든다.

# GridSearch
# GridSearch는 우리가 지정해준 몇 가지 잠재적인 파라미터들의 후보군들의 조합 중에서 가장 beat 조합을 찾아준다.
# 사이킷런 패키지가 제공하므로 하나하나 대입해 가면서 loss를 확인하는 작업을 GridSearch가 대신 해주므로 손쉽게
# 사용할 수 있다.
# 가장 큰 단점은 하이퍼파라미터 후보군의 갯수만큼 비례해서 시간이 늘어나 최적의 조합을 찾는 시간이 매우 오래
# 걸린다는 단점이 있다.
from sklearn.model_selection import GridSearchCV # 그리드 서치 알고리즘을 사용하기 위해 import 한다.
# 그리드 서치에 적용할 하이퍼파라미터 후보군을 설정한다.
param_grid = {
    'kernel': ('linear', 'rbf'), # 서포트 벡터 머신에서 커널은 정확도가 높은 linear 또는 rbf로 설정한다.
    'C': [0.5, 1, 10, 100]
}

# n_splits 옵션은 트레이닝 데이터를 나룰 개수를 지정하고 shuffle=True 옵션을 지정하면 데이터를 섞는다.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0) # 크로스 벨리데이션 객체를 만든다.
# estimator 옵션으로 학습할 모형을 지정하고 param_grid 옵션으로 하이퍼파라미터 후보군을 지정하고 cv 옵션으로
# 크로스 밸리데이션 방법을 지정하고 scoring 옵션으로 모형 평가 방법을 지정해서 그리드 서치 객체를 만든다.
grid_cv = GridSearchCV(estimator=clf_svm, param_grid=param_grid, cv=kfold, scoring='accuracy')
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 그리드 서이 알고리즘을 학습시킨다.
grid_cv.fit(X_tn_std, y_tn)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(random_state=0),
             param_grid={'C': [0.5, 1, 10, 100], 'kernel': ('linear', 'rbf')},
             scoring='accuracy')

In [7]:
# 그리드 서치 결과 확인
# 그리드 서치 객체에서 cv_results_ 속성으로 그리드 서치 결과(설정한 파라미터, 모형 평가 등)를 확인할 수 있다.
print(grid_cv.cv_results_)

{'mean_fit_time': array([0.00079999, 0.00100036, 0.0006001 , 0.00100031, 0.00120039,
       0.00100017, 0.00074849, 0.00100012]), 'std_fit_time': array([3.99995316e-04, 3.50402318e-07, 4.89979892e-04, 4.42200589e-07,
       4.00328647e-04, 4.76837158e-07, 3.87408538e-04, 2.33601546e-07]), 'mean_score_time': array([0.00039968, 0.00040007, 0.00020008, 0.00039997, 0.        ,
       0.        , 0.00020013, 0.00020008]), 'std_score_time': array([0.00048951, 0.00048998, 0.00040016, 0.00048986, 0.        ,
       0.        , 0.00040026, 0.00040016]), 'param_C': masked_array(data=[0.5, 0.5, 1, 1, 10, 10, 100, 100],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf',
                   'linear', 'rbf'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params':

In [8]:
# 그리드 서치 결과 확인 - 데이터프레임
import numpy as np
import pandas as pd
pd.DataFrame(grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000800,3.999953e-04,0.0004,0.00049,0.5,linear,"{'C': 0.5, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
1,0.001000,3.504023e-07,0.0004,0.00049,0.5,rbf,"{'C': 0.5, 'kernel': 'rbf'}",0.962963,1.000000,0.962963,0.961538,1.000000,0.977493,0.018384,1
2,0.000600,4.899799e-04,0.0002,0.00040,1,linear,"{'C': 1, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
3,0.001000,4.422006e-07,0.0004,0.00049,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2
4,0.001200,4.003286e-04,0.0000,0.00000,10,linear,"{'C': 10, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
5,0.001000,4.768372e-07,0.0000,0.00000,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2
6,0.000748,3.874085e-04,0.0002,0.00040,100,linear,"{'C': 100, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,5
7,0.001000,2.336015e-07,0.0002,0.00040,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,2


In [9]:
np.transpose(pd.DataFrame(grid_cv.cv_results_))

,0,1,2,3,4,5,6,7
mean_fit_time,0.00079999,0.00100036,0.0006001,0.00100031,0.00120039,0.00100017,0.000748491,0.00100012
std_fit_time,0.000399995,3.50402e-07,0.00048998,4.42201e-07,0.000400329,4.76837e-07,0.000387409,2.33602e-07
mean_score_time,0.000399685,0.000400066,0.000200081,0.000399971,0,0,0.000200129,0.000200081
std_score_time,0.000489512,0.000489979,0.000400162,0.000489862,0,0,0.000400257,0.000400162
param_C,0.5,0.5,1,1,10,10,100,100
param_kernel,linear,rbf,linear,rbf,linear,rbf,linear,rbf
params,"{'C': 0.5, 'kernel': 'linear'}","{'C': 0.5, 'kernel': 'rbf'}","{'C': 1, 'kernel': 'linear'}","{'C': 1, 'kernel': 'rbf'}","{'C': 10, 'kernel': 'linear'}","{'C': 10, 'kernel': 'rbf'}","{'C': 100, 'kernel': 'linear'}","{'C': 100, 'kernel': 'rbf'}"
split0_test_score,0.888889,0.962963,0.888889,0.925926,0.888889,0.925926,0.888889,0.925926
split1_test_score,0.962963,1,0.962963,0.962963,0.962963,0.962963,0.962963,0.962963
split2_test_score,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963


In [10]:
# 베스트 스코어와 베스트 스코어를 얻어낸 하이퍼파라미터를 확인한다.
print(grid_cv.best_score_) # 그리드 서치 결과 중 베스트 스코어를 얻어온다.
print(grid_cv.best_params_) # 그리드 서치 결과 중 베스트 스코어를 얻어낸 하이퍼파라미터를 얻어온다.

0.9774928774928775
{'C': 0.5, 'kernel': 'rbf'}


In [11]:
# 최종 모형
clf_grid = grid_cv.best_estimator_ # 그리드 서치를 통해 구한 베스트 모델을 최종 모델로 설정한다.
print(clf_grid)

SVC(C=0.5, random_state=0)


In [17]:
# 크로스 밸리데이션 스코어 확인 - 1
from sklearn.model_selection import cross_validate # cross_validate 확인에 사용하기 위해 import 한다.
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'] # 모델 평가 방법을 설정한다.
cv_score = cross_validate(clf_grid, X_tn_std, y_tn, cv=kfold, scoring=metrics)
for key in cv_score:
    print(key, cv_score[key])

fit_time [0.00099993 0.00099993 0.00099969 0.0010004  0.0010004 ]
score_time [0.00200057 0.00200081 0.00200081 0.00200057 0.00099993]
test_accuracy [0.96296296 1.         0.96296296 0.96153846 1.        ]
test_precision_macro [0.96296296 1.         0.96969697 0.96969697 1.        ]
test_recall_macro [0.96666667 1.         0.96296296 0.95833333 1.        ]
test_f1_macro [0.9628483  1.         0.96451914 0.96190476 1.        ]


In [24]:
# 크로스 밸리데이션 스코어 확인 - 2
# cross_validate 함수가 여러 개의 지표를 동시에 확인하는데 사용되었다면 cross_val_score 함수는 한 가지 지표를
# 확인하는데 사용된다.
from sklearn.model_selection import cross_val_score # cross_validate 확인에 사용하기 위해 import 한다.
cv_score = cross_val_score(clf_grid, X_tn_std, y_tn, cv=kfold, scoring='accuracy')
print(cv_score) # 각 split별 정확도 확인
print(cv_score.mean()) # 각 split별 정확도의 평균 확인
print(cv_score.std()) # 각 split별 정확도의 표준편차 확인

[0.96296296 1.         0.96296296 0.96153846 1.        ]
0.9774928774928775
0.01838434849561446


In [26]:
# 데이터 예측
pred_svm = clf_grid.predict(X_te_std) # 표준화된 테스트 데이터 X_te_std로 예측한다.
print(pred_svm)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [27]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 정확도를 평가하기 위해 import 한다.
# accuracy_score 함수의 인수로 실제 타겟 데이터와 예측된 데이터를 넘겨 정확도를 평가한다.
accuracy = accuracy_score(y_te, pred_svm)
print(accuracy)

1.0


In [28]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_svm)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


In [29]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_svm)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

